# スタサプ RPA

## モジュール読み込み

In [51]:
# selenium 4
import math
import time
import os
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.wait import TimeoutException
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import selenium.webdriver.common.devtools.v114 as devtools
from selenium.webdriver.common.by import By
# .env file load
from dotenv import load_dotenv
load_dotenv()

True

## インスタンスの生成
### 実行

In [46]:
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')

driver = webdriver.Firefox(service=FirefoxService(
    GeckoDriverManager().install()), options=options)
driver.set_window_position(0, 0)

original_window = driver.current_window_handle

### .env 読み込み

In [47]:
service_url = os.getenv("LOGIN_URL")
email = os.getenv("EMAIL_ADDRESS")
password = os.getenv("PASSWORD")

### ログイン

In [48]:
driver.get(service_url)
username_input_element = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[2]/input')
password_input_element = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[3]/span/input')
login_button = driver.find_element(
    by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/button[2]/span[2]')

username_input_element.clear()
password_input_element.clear()
username_input_element.send_keys(email)
password_input_element.send_keys(password)
login_button.click()

time.sleep(2)

### URL を設定

In [49]:
studysapuri_uri_dict = {
    "homework_active": "https://learn.studysapuri.jp/ja/todos/active",
    "homework_expired": "https://learn.studysapuri.jp/ja/todos/expired"
}


### 課題状況検査

課題の残りの数をカウントする

In [57]:
driver.get(studysapuri_uri_dict["homework_active"])
WebDriverWait(driver, timeout=4).until(
    lambda d: d.find_elements(
        by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
)

try:
    is_active_homework_empty = WebDriverWait(driver, timeout=4).until(
        lambda d: d.find_elements(
            by=By.CSS_SELECTOR, value="div > ul li > button")
    )
except TimeoutException as err:
    is_active_homework_empty = []

# -------------------------------------------------------------------

driver.get(studysapuri_uri_dict["homework_expired"])
WebDriverWait(driver, timeout=4).until(
    lambda d: d.find_elements(
        by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
)

try:
    is_expired_homework_empty = WebDriverWait(driver, timeout=4).until(
        lambda d: d.find_elements(
            by=By.CSS_SELECTOR, value="div > ul li > button")
    )
except TimeoutException as err:
    is_expired_homework_empty = []

残りの課題の数を表示

In [58]:
print(len(is_active_homework_empty),len(is_expired_homework_empty))


0 7


最初の課題を実行

In [59]:
first_taskwork = driver.find_element(
    by=By.CSS_SELECTOR, value='button[class*="TodoCard"]')
first_taskwork.click()

first_todo = driver.find_element(
    by=By.CSS_SELECTOR, value="div > ul li div > ul > li")
first_todo.click()

In [60]:
def quession_automation() -> None:
    """確認問題をランダムに解く
    """
    driver.execute_script("""
    var shuffleArray = (array) => {
            const cloneArray = [...array];
            for (let i = cloneArray.length - 1; i >= 0; i--) {
                let rand = Math.floor(Math.random() * (i + 1));
                let tmpStorage = cloneArray[i];
                cloneArray[i] = cloneArray[rand];
                cloneArray[rand] = tmpStorage;
            }
            return cloneArray;
        };
        
        setTimeout(() => {
            shuffleArray(document.querySelectorAll("[class*=TopicsPage__Main]  button") )
                .forEach(each => each.click());
        },1000)
        setTimeout(()=>{
            document.querySelector('button[class*=RaisedButton]').click();
        },1500)
    """)
    find_element = driver.find_elements(
        by=By.CSS_SELECTOR, value="[class*=RaisedButton]")

    while(not driver.current_url.endswith("result")):
        quession_automation()
        time.sleep(0.5)
    print("Quiz compreat.")

各チャプタのタブを開く

In [61]:
WebDriverWait(driver, timeout=4).until(lambda d: d.find_element(
    by=By.CSS_SELECTOR, value='ul[class*="LessonStepList"] > li > a')
)

after_second_lesston_list = map(lambda element: element.get_attribute("href"), driver.find_elements(
    by=By.CSS_SELECTOR, value='ul[class*="LessonStepList"] > li > a'))

devtools.network.set_blocked_ur_ls(
    urls=["https://mediacdn.studysapuri.jp/media/**/*"])
i = 0

try:

    for href in list(after_second_lesston_list):
        driver.switch_to.new_window("tab")
        driver.get(href)

        if "questions" in href.split("/"):
            print(href)
            quession_automation()

        driver.execute_script("""
            setTimeout(() => {
                document.querySelector("video").volume = 0;
                document.querySelector("[class*=hugereplaybutton]").click();
                document.querySelector("video").play();
                document.querySelector("video").playbackRate = 16;
                console.log("RUN");
            },1700)
            """)

except:
    pass
finally:
    driver.switch_to.window(original_window)


https://learn.studysapuri.jp/ja/assignments/64738a6eb2a3f42a31765384/topics/578dd8f39c1cfd001a0001b1/questions/10
Quiz compreat.


In [43]:
driver.switch_to.window(original_window)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:468:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.getWindowHandle@chrome://remote/content/marionette/driver.sys.mjs:1026:15
despatch@chrome://remote/content/marionette/server.sys.mjs:304:40
execute@chrome://remote/content/marionette/server.sys.mjs:275:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:248:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:249:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:496:20


In [44]:
driver.quit() # 終了